<a href="https://colab.research.google.com/github/Palaeoprot/ProteoParc_Colab/blob/main/PalaeoParc_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##License

The [ProteoParc](https://github.com/guillecarrillo/proteoparc) pipeline was written by [Guillermo Carrillo Martín](https://github.com/guillecarrillo) and this is his license from Github. "[Palaeoparc] "*is the result of many collective work, so its citation in any study in which it is used would be appreciated.


 Every user is free to change any section of the code so it can be adjusted to individual necessities, if you do so, please indicate it in your manuscripts writing the changes to avoid confusion. For more information, read the License This pi[link text](https://)peline has been possible thanks to a lot of collective work, so its citation in any study in which it is used would be appreciated. Every user is free to change any section of the code so it can be adjusted to individual necessities, if you do so, please indicate it in your manuscripts to avoid confusion.*"

Here is my take on it in Colab

<img src='https://drive.google.com/uc?export=view&id=1FqSr3E9UQKtRO4KwR6LjAU8TPrD8Oq6q' width=800px align=centre>





In [ ]:
!pip install biopython
import requests, sys, json, os, time  #data handling
import argparse  # command-line arguments
import shlex  # parsing shell commands
import subprocess  # For system commands (already imported twice)
import ipywidgets as widgets  # For interactive elements
from IPython.display import display, HTML, clear_output  # For interactive output
import shutil
import re
import os
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from requests.adapters import HTTPAdapter, Retry
from datetime import datetime

# Mount Google Drive (Optional, you can comment out if not using)
from google.colab import drive
drive.mount('/content/drive')

# Global constants
WEBSITE_API = "https://rest.uniprot.org/"

In [ ]:
bone_proteins = {
    "ALB": "serum albumin",
    "AEBP1": "AE binding protein 1",
    "AHSG": "alpha-2-HS-glycoprotein (fetuin-A)",
    "ALPL": "alkaline phosphatase, liver/bone/kidney",
    "APOA1": "apolipoprotein A-I",
    "APOA4": "apolipoprotein A-IV",
    "APOE": "apolipoprotein E",
    "APOC1": "apolipoprotein C-I",
    "APP": "amyloid beta precursor protein",
    "ASPN": "asporin",
    "BGLAP": "bone gamma-carboxyglutamate protein (osteocalcin)",
    "BGN": "biglycan",
    "C3": "complement component 3",
    "C8B": "complement component 8, beta polypeptide",
    "C9": "complement component 9",
    "CFH": "complement factor H",
    "CHAD": "chondroadherin",
    "CHGA": "chromogranin A",
    "CLEC11A": "C-type lectin domain family 11, member A",
    "CLEC3B": "C-type lectin domain family 3, member B",
    "CLU": "clusterin",
    "COL10A1": "collagen type X alpha 1 chain",
    "COL11A1": "collagen type XI alpha 1 chain",
    "COL11A2": "collagen type XI alpha 2 chain",
    "COL12A1": "collagen type XII alpha 1 chain",
    "COL16A1": "collagen type XVI alpha 1 chain",
    "COL1A1": "collagen type I alpha 1 chain",
    "COL1A2": "collagen type I alpha 2 chain",
    "COL21A1": "collagen type XXI alpha 1 chain",
    "COL22A1": "collagen type XXII alpha 1 chain",
    "COL2A1": "collagen type II alpha 1 chain",
    "COL3A1": "collagen type III alpha 1 chain",
    "COL4A3": "collagen type IV alpha 3 chain",
    "COL4A4": "collagen type IV alpha 4 chain",
    "COL4A5": "collagen type IV alpha 5 chain",
    "COL5A1": "collagen type V alpha 1 chain",
    "COL5A2": "collagen type V alpha 2 chain",
    "COL5A3": "collagen type V alpha 3 chain",
    "COL6A1": "collagen type VI alpha 1 chain",
    "COL6A3": "collagen type VI alpha 3 chain",
    "COL8A1": "collagen type VIII alpha 1 chain",
    "CRP": "C-reactive protein",
    "DCN": "decorin",
    "DPT": "dermatopontin",
    "EEF1A1": "eukaryotic translation elongation factor 1 alpha 1",
    "EMILIN1": "elastin microfibril interfacer 1",
    "EZR": "ezrin",
    "F10": "coagulation factor X",
    "F2": "coagulation factor II (thrombin)",
    "F7": "coagulation factor VII",
    "F9": "coagulation factor IX",
    "FGL2": "fibrinogen-like 2",
    "FMOD": "fibromodulin",
    "FN1": "fibronectin 1",
    "GAPDH": "glyceraldehyde-3-phosphate dehydrogenase",
    "GAS6": "growth arrest-specific 6",
    "GC": "group-specific component (vitamin D binding protein)",
    "HAPLN3": "hyaluronan and proteoglycan link protein 3",
    "HSP90B1": "heat shock protein 90 beta family member 1",
    "HSPA5": "heat shock protein family A (Hsp70) member 5",
    "HTRA1": "HtrA serine peptidase 1",
    "IBSP": "integrin binding sialoprotein",
    "IGF1": "insulin-like growth factor 1",
    "IGF2": "insulin-like growth factor 2",
    "IGFALS": "insulin-like growth factor binding protein, acid labile subunit",
    "IGFBP1": "insulin-like growth factor binding protein 1",
    "KNG1": "kininogen 1",
    "KRT2": "keratin 2",
    "LOX": "lysyl oxidase",
    "LRRC15": "leucine rich repeat containing 15",
    "LUM": "lumican",
    "MGP": "matrix Gla protein",
    "MMP2": "matrix metallopeptidase 2",
    "MSN": "moesin",
    "MYO1B": "myosin IB",
    "NUCB1": "nucleobindin 1",
    "NUCB2": "nucleobindin 2",
    "OGN": "osteoglycin",
    "OLFML1": "olfactomedin-like 1",
    "OLFML3": "olfactomedin-like 3",
    "OMD": "osteomodulin",
    "P4HB": "prolyl 4-hydroxylase subunit beta",
    "PAM": "peptidylglycine alpha-amidating monooxygenase",
    "PCOLCE": "procollagen C-endopeptidase enhancer",
    "PHOSPHO1": "phosphatase, orphan 1",
    "POSTN": "periostin",
    "PROC": "protein C, inactivator of coagulation factors Va and VIIIa",
    "PROS1": "protein S, vitamin K-dependent",
    "PRSS2": "protease, serine 2",
    "SERPINC1": "serpin family C member 1 (antithrombin)",
    "SERPIND1": "serpin family D member 1 (heparin cofactor II)",
    "SERPINF1": "serpin family F member 1",
    "SLC8A3": "solute carrier family 8 member A3",
    "SPARC": "secreted protein acidic and rich in cysteine (osteonectin)",
    "SPARCL1": "SPARC-like 1 (hevin)",
    "SPP1": "secreted phosphoprotein 1 (osteopontin)",
    "SPP2": "secreted phosphoprotein 2",
    "TGFB1": "transforming growth factor beta 1",
    "THBS1": "thrombospondin 1",
    "TNC": "tenascin C",
    "TPP1": "tripeptidyl peptidase I",
    "TUBA1B": "tubulin alpha 1b",
    "VCAN": "versican",
    "VIT": "vitrin",
    "VTN": "vitronectin",
    "BANP": "BTG3-associated nuclear protein",
    "BCL9L": "B-cell CLL/lymphoma 9-like protein",
    "CD9": "CD9 molecule",
    "CD36": "CD36 molecule (thrombospondin receptor)",
    "CD63": "CD63 molecule",
    "CD81": "CD81 molecule",
    "COL4A1": "collagen type IV alpha 1 chain",
    "COL4A2": "collagen type IV alpha 2 chain",
    "COL6A5": "collagen type VI alpha 5 chain",
    "COL8A2": "collagen type VIII alpha 2 chain",
    "COL9A1": "collagen type IX alpha 1 chain",
    "COL9A2": "collagen type IX alpha 2 chain",
    "COL17A1": "collagen type XVII alpha 1 chain",
    "COL18A1": "collagen type XVIII alpha 1 chain",
    "COL26A1": "collagen type XXVI alpha 1 chain",
    "COL28A1": "collagen type XXVIII alpha 1 chain",
    "EMID1": "EMI domain containing 1",
    "IGHA2": "immunoglobulin heavy constant alpha 2",
    "LTF": "lactotransferrin",
    "LYZ": "lysozyme",
    "PIP": "prolactin-induced protein",
    "S100A7A": "S100 calcium binding protein A7A",
    "S100A9": "S100 calcium binding protein A9",
    "TLR3": "toll like receptor 3",
    "TMEM119": "transmembrane protein 119"
}

In [ ]:

enamel_proteins = {
    "AHSG": "alpha-2-HS-glycoprotein (fetuin-A)",
    "ALB": "serum albumin",
    "AMBN": "ameloblastin",
    "AMEL": "amelogenin (unspecified isoform)",
    "AMELY": "amelogenin Y-linked",
    "AMELX": "amelogenin X-linked",
    "AMTN": "amelotin",
    "COL1A1": "collagen type I alpha 1 chain",
    "COL1A2": "collagen type I alpha 2 chain",
    "COL17A1": "collagen type XVII alpha 1 chain",
    "COL2A1": "collagen type II alpha 1 chain",
    "DSPP": "dentin sialophosphoprotein",
    "ENAM": "enamelin",
    "KLK4": "kallikrein related peptidase 4",
    "MMP20": "matrix metallopeptidase 20",
    "ODAM": "odontogenic, ameloblast associated",
    "SERPINC1": "serpin family C member 1 (antithrombin)",
    "TUFT1": "tuftelin 1"
}

food_crust_proteins = [
    ("CSN1S1", "alpha-S1 casein"),
    ("CSN1S2", "alpha-S2 casein"),
    ("CSN2", "beta-casein"),
    ("CSN3", "kappa-casein"),
    ("LALBA", "alpha-lactalbumin"),
    ("LGB", "beta-lactoglobulin"),
    ("IGH@", "immunoglobulin heavy chains"),
    ("IGK@", "immunoglobulin kappa light chains"),
    ("IGL@", "immunoglobulin lambda light chains"),
    ("LTF", "lactoferrin"),
    ("LYZ", "lysozyme"),
    ("ALB", "serum albumin"),
    ("MUC1", "mucin 1"),
    ("BTN1A1", "butyrophilin subfamily 1 member A1"),
    ("XDH", "xanthine dehydrogenase/oxidase"),
    ("MFGE8", "milk fat globule-EGF factor 8 protein")
]

saliva_proteins = [
    # Digestive Enzymes
    ("AMY1", "Alpha-amylase"),
    ("LING1", "Lingual lipase"),
    # Antimicrobial Proteins
    ("LYZ", "Lysozyme"),
    ("LPO", "Lactoperoxidase"),
    ("MUC7", "Mucin-7"),
    ("MUC5B", "Mucin-5B"),
    ("DMBT1", "Deleted in malignant brain tumors 1"),
    ("S100A7", "Psoriasin"),
    ("S100A8", "Calprotectin"),
    ("S100A9", "Calprotectin"),
    ("SLPI", "Secretory leukocyte protease inhibitor"),
    ("BPIFA1", "BPI fold-containing family A member 1"),
    ("BPIFB2", "BPI fold-containing family B member 2"),
    # Immunoglobulin and Immune System-Related Proteins
    ("IGHG1", "Immunoglobulin heavy constant gamma 1"),
    ("IGHG2", "Immunoglobulin heavy constant gamma 2"),
    ("IGHG3", "Immunoglobulin heavy constant gamma 3"),
    ("IGHG4", "Immunoglobulin heavy constant gamma 4"),
    ("IGHA1", "Immunoglobulin heavy constant alpha 1"),
    ("IGHA2", "Immunoglobulin heavy constant alpha 2"),
    ("IGKC", "Immunoglobulin kappa constant"),
    ("IGLC1", "Immunoglobulin lambda constant 1"),
    ("IGLC2", "Immunoglobulin lambda constant 2"),
    ("IGLC3", "Immunoglobulin lambda constant 3")
]

milk_proteins = [
    ("CSN1S1", "alpha-S1-casein"),
    ("CSN2", "beta-casein"),
    ("CSN3", "kappa-casein"),
    ("CSN1S2", "alpha-S2-casein"),
    ("LALBA", "alpha-lactalbumin"),
    ("BLG", "beta-lactoglobulin"),
    ("PAEP", "Pregnancy-associated glycoprotein"),
    ("LCN2", "Lactoferrin"),
    ("LTF", "Lactotransferrin"),
    ("ALB", "Serum albumin"),
    ("HP", "Haptoglobin"),
    ("IGLC1", "Immunoglobulin lambda constant 1"),
    ("IGLC2", "Immunoglobulin lambda constant 2"),
    ("IGLC3", "Immunoglobulin lambda constant 3"),
    ("IGHA1", "Immunoglobulin alpha 1"),
    ("IGHA2", "Immunoglobulin alpha 2"),
    ("IGHG1", "Immunoglobulin gamma 1"),
    ("IGHG2", "Immunoglobulin gamma 2"),
    ("IGHG3", "Immunoglobulin gamma 3"),
    ("IGHG4", "Immunoglobulin gamma 4"),
    ("MFGE8", "Milk fat globule-EGF factor 8"),
    ("BTN1A1", "Butyrophilin subfamily 1 member A1"),
    ("XDH", "Xanthine dehydrogenase"),
    ("AZGP1", "Zinc-alpha-2-glycoprotein")
]

muscle_proteins = [
    ("ACTB", "Actin, cytoplasmic 1"),
    ("ATP2A1", "Sarcoplasmic/endoplasmic reticulum calcium ATPase 1"),
    ("ATP5F1B", "ATP synthase subunit beta, mitochondrial precursor"),
    ("ATP6", "ATP synthase subunit a"),
    ("CACNA1S", "Calcium Voltage-Gated Channel Subunit Alpha1 S"),
    ("CAMK2G", "Calcium/Calmodulin Dependent Protein Kinase II Gamma"),
    ("CKMT2", "Creatine kinase, testis isozyme"),
    ("COL1A1", "Collagen type I alpha 1 chain"),
    ("COL1A2", "Collagen type I alpha 2 chain"),
    ("COL2A1", "Collagen type II alpha 1 chain"),
    ("COL3A1", "Collagen type III alpha 1 chain"),
    ("COL5A1", "Collagen type V alpha 1 chain"),
    ("COL5A2", "Collagen type V alpha 2 chain"),
    ("CS", "Citrate synthase, mitochondrial precursor"),
    ("ELOVL5", "ELOVL Fatty Acid Elongase 5"),
    ("FABP3", "Fatty Acid Binding Protein 3"),
    ("FABP4", "Fatty Acid Binding Protein 4"),
    ("FASN", "Fatty Acid Synthase"),
    ("HBA", "Hemoglobin cathodic subunit alpha"),
    ("HBB", "Hemoglobin subunit beta"),
    ("LDHA", "L-lactate dehydrogenase A chain"),
    ("LDHB", "L-lactate dehydrogenase B chain"),
    ("MB", "Myoglobin"),
    ("MC4R", "Melanocortin 4 Receptor"),
    ("MYH4", "Myosin heavy chain, fast skeletal muscle"),
    ("MYH6", "Atrial myosin heavy chain"),
    ("MYH9A", "Myosin-9a"),
    ("MYH11A", "Myosin-11a"),
    ("MYLPFA", "Myosin regulatory light chain 2, skeletal muscle isoform A"),
    ("PPP1R12A", "Protein phosphatase 1 regulatory subunit 12A"),
    ("MYLK3", "Myosin light chain kinase 3"),
    ("MYO6A", "Unconventional myosin-VI"),
    ("TPI1", "Triosephosphate isomerase"),
    ("TPM1", "Tropomyosin-1 alpha chain"),
    ("MYH9B", "Myosin-9b"),
    ("MYL1", "Myosin light chain 3, skeletal muscle isoform"),
    ("MYLIPA", "E3 ubiquitin-protein ligase MYLIP-A"),
    ("MYO1C", "Unconventional myosin-Ic"),
    ("MYO1D", "Unconventional myosin-Id"),
    ("MYO5B", "Myosin VB"),
    ("MYO6B", "Unconventional myosin-VI"),
    ("MYO9AA", "Unconventional myosin-IXAa"),
    ("MYO9AB", "Unconventional myosin-IXAb"),
    ("NDPKB", "Nucleoside diphosphate kinase B"),
    ("PDE3B", "Phosphodiesterase 3B"),
    ("PRKAG3", "Protein Kinase AMP-Activated Non-Catalytic Subunit Gamma 3"),
    ("PVALB", "Parvalbumin beta"),
    ("PVALB2", "Parvalbumin-2"),
    ("PVALB7", "Parvalbumin-7"),
    ("PVALB8", "Parvalbumin-8"),
    ("PVALB9", "Parvalbumin-9"),
    ("RYR1", "Ryanodine Receptor 1")
]

blood_proteins = [
    ("ALB", "Albumin"),
    ("HBA", "Hemoglobin subunit alpha"),
    ("HBB", "Hemoglobin subunit beta"),
    ("TTR", "Transthyretin"),
    ("APOA1", "Apolipoprotein A-I"),
    ("APOB", "Apolipoprotein B"),
    ("APOC3", "Apolipoprotein C-III"),
    ("TF", "Transferrin"),
    ("IGG1", "Immunoglobulin G1"),
    ("IGG2", "Immunoglobulin G2"),
    ("IGG3", "Immunoglobulin G3"),
    ("IGG4", "Immunoglobulin G4"),
    ("IGA1", "Immunoglobulin A1"),
    ("IGA2", "Immunoglobulin A2"),
    ("IGM", "Immunoglobulin M"),
    ("FGA", "Fibrinogen alpha chain"),
    ("FGB", "Fibrinogen beta chain"),
    ("FGG", "Fibrinogen gamma chain"),
    ("CRP", "C-reactive protein"),
    ("CP", "Ceruloplasmin"),
    ("HP", "Haptoglobin"),
    ("GC", "Vitamin D-binding protein"),
    ("AHSG", "Alpha-2-HS-glycoprotein"),
    ("C3", "Complement C3"),
    ("C4A", "Complement C4A"),
    ("C4B", "Complement C4B"),
    ("C5", "Complement C5"),
    ("SAA1", "Serum amyloid A1"),
    ("SAA2", "Serum amyloid A2"),
    ("KLK3", "Prostate-specific antigen"),
    ("AMBP", "Alpha-1-microglobulin/bikunin precursor"),
    ("ORM1", "Orosomucoid 1"),
    ("ORM2", "Orosomucoid 2"),
    ("SELM", "Selenoprotein M"),
    ("SELP", "P-selectin"),
    ("SELE", "E-selectin"),
    ("SELL", "L-selectin"),
    ("VWF", "Von Willebrand factor"),
    ("PLG", "Plasminogen"),
    ("F12", "Coagulation factor XII"),
    ("F11", "Coagulation factor XI"),
    ("F9", "Coagulation factor IX"),
    ("F8", "Coagulation factor VIII"),
    ("F7", "Coagulation factor VII"),
    ("F10", "Coagulation factor X"),
    ("F5", "Coagulation factor V"),
    ("PROC", "Protein C"),
    ("PROS1", "Protein S"),
    ("THBD", "Thrombomodulin"),
    ("ATF3", "Activating transcription factor 3"),
    ("SERPINA1", "Alpha-1-antitrypsin"),
    ("SERPINA3", "Alpha-1-antichymotrypsin")
]

skin_keratins =  [
        ("KRT1", "Keratin, type II cytoskeletal 1"),
        ("KRT2", "Keratin, type II cytoskeletal 2"),
        ("KRT3", "Keratin, type II cytoskeletal 3"),
        ("KRT4", "Keratin, type II cytoskeletal 4"),
        ("KRT5", "Keratin, type II cytoskeletal 5"),
        ("KRT6A", "Keratin, type II cytoskeletal 6A"),
        ("KRT6B", "Keratin, type II cytoskeletal 6B"),
        ("KRT6C", "Keratin, type II cytoskeletal 6C"),
        ("KRT9", "Keratin, type I cytoskeletal 9"),
        ("KRT10", "Keratin, type I cytoskeletal 10"),
        ("KRT13", "Keratin, type I cytoskeletal 13"),
        ("KRT14", "Keratin, type I cytoskeletal 14"),
        ("KRT15", "Keratin, type I cytoskeletal 15"),
        ("KRT16", "Keratin, type I cytoskeletal 16"),
        ("KRT24", "Keratin, type I cytoskeletal 24"),
        ("KRT31", "Keratin, type I cytoskeletal 31"),
        ("KRT32", "Keratin, type I cytoskeletal 32"),
        ("KRT33B", "Keratin, type I cytoskeletal 33B"),
        ("KRT34", "Keratin, type I cytoskeletal 34"),
        ("KRT37", "Keratin, type I cytoskeletal 37"),
        ("KRT38", "Keratin, type I cytoskeletal 38"),
        ("KRT74", "Keratin, type II cytoskeletal 74"),
        ("KRT77", "Keratin, type II cytoskeletal 77"),
        ("KRT78", "Keratin, type II cytoskeletal 78"),
        ("KRT82", "Keratin, type II cytoskeletal 82"),
        ("KRT83", "Keratin, type II cytoskeletal 83"),
        ("KRT85", "Keratin, type II cytoskeletal 85"),
        ("KRT86", "Keratin, type II cytoskeletal 86")
]

hair_keratins  =  [
        ("KRT31", "Keratin, type I cytoskeletal 31"),
        ("KRT32", "Keratin, type I cytoskeletal 32"),
        ("KRT33A", "Keratin, type I cytoskeletal 33A"),
        ("KRT33B", "Keratin, type I cytoskeletal 33B"),
        ("KRT34", "Keratin, type I cytoskeletal 34"),
        ("KRT35", "Keratin, type I cytoskeletal 35"),
        ("KRT36", "Keratin, type I cytoskeletal 36"),
        ("KRT37", "Keratin, type I cytoskeletal 37"),
        ("KRT38", "Keratin, type I cytoskeletal 38"),
        ("KRT39", "Keratin, type I cytoskeletal 39"),
        ("KRT40", "Keratin, type I cytoskeletal 40"),
        ("KRT81", "Keratin, type II cytoskeletal 81"),
        ("KRT83", "Keratin, type II cytoskeletal 83"),
        ("KRT85", "Keratin, type II cytoskeletal 85"),
        ("KRT86", "Keratin, type II cytoskeletal 86"),
        # High sulfur proteins"
        ("KAP1", "Keratin-associated protein 1"),
        ("KAP2", "Keratin-associated protein 2"),
        ("KAP3", "Keratin-associated protein 3"),
        ("KAP4", "Keratin-associated protein 4"),
        ("KAP5", "Keratin-associated protein 5"),
        ("KAP6", "Keratin-associated protein 6"),
        ("KAP7", "Keratin-associated protein 7"),
        ("KAP8", "Keratin-associated protein 8"),
        ("KAP9", "Keratin-associated protein 9"),
        ("KAP10", "Keratin-associated protein 10"),
        ("KAP11", "Keratin-associated protein 11"),
        ("KAP12", "Keratin-associated protein 12"),
        ("KAP13", "Keratin-associated protein 13"),
        ("KAP14", "Keratin-associated protein 14"),
        ("KAP15", "Keratin-associated protein 15"),
        ("KAP16", "Keratin-associated protein 16"),
        ("KAP17", "Keratin-associated protein 17"),
        ("KAP18", "Keratin-associated protein 18"),
        ("KAP19", "Keratin-associated protein 19"),
        ("KAP20", "Keratin-associated protein 20")
]


zooms_genes = [
    ("COL1A2", "collagen type I alpha 2 chain"),
    ("COL1A1", "collagen type I alpha 1 chain"),
    ("COL3A1", "collagen type III alpha 1 chain")
]

# Define a combined dictionary of all genes
all_genes = {
    "Bone": bone_proteins,
    "Enamel": enamel_proteins,
    "Food Crust": food_crust_proteins,
    "Muscle": muscle_proteins,
    "Blood": blood_proteins,
    "Skin": skin_keratins,
    "Hair": hair_keratins,
    "Saliva": saliva_proteins,
    "Milk": milk_proteins,
    "Zooms": zooms_genes
}

# Define the taxonomy IDs and their names
taxonomy_dict = {
    # Carnivora
    33554: "Carnivora", 379584: "Caniformia", 379583: "Feliformia", 9632: "Ursidae", 9615: "Canis lupus familiaris", 9647: "Procyonidae",

    # Bovidae
    35500: "Pecora", 9850: "Cervidae", 9913: "Bovidae", 9895: "Bos taurus", 9925: "Capra hircus", 9940: "Ovis aries",

    # Lagomorpha
    9975: "Lagomorpha",

    # Humans
    9606: "Homo sapiens", 9605: "Homo", 9604: "Hominidae", 9443: "Primates", 40674: "Mammalia", 7711: "Chordata", 33208: "Animalia",

    # Ostriches
    8839: "Struthio camelus", 8838: "Struthio", 8837: "Struthionidae", 30498: "Struthioniformes", 8782: "Aves",

    # Cod
    8049: "Gadus morhua", 8048: "Gadus", 27502: "Gadidae", 27682: "Gadiformes", 7898: "Actinopterygii",

    # Geckos
    103697: "Gekko gecko", 8489: "Gekko", 8483: "Gekkonidae", 8509: "Squamata", 8457: "Reptilia"
}

# print("all_genes:")
# for tissue, genes in all_genes.items():
#     print(f"  {tissue}: {len(genes)} genes")

# print("\ntaxonomy_dict:")
# for tax_id, tax_name in taxonomy_dict.items():
#     print(f"  {tax_id}: {tax_name}")

In [ ]:
#@title #Functions

def api_taxid_descendent_list(tax_id):
    """Gets all descendant taxonomy IDs for a given tax_id"""
    print(f"\nGetting taxonomy descendants for TaxID: {tax_id}")
    url_tax_id_descendent = f"https://rest.uniprot.org/taxonomy/stream?format=list&query=%28%28ancestor%3A{str(tax_id)}%29%29"
    tax_id_download = str(requests.get(url_tax_id_descendent).text)
    tax_id_descendent_list = tax_id_download.split("\n")
    tax_id_descendent_list.pop()  # Remove empty last element
    tax_id_descendent_list = [int(taxid) for taxid in tax_id_descendent_list]
    tax_id_descendent_list.append(int(tax_id))
    return tax_id_descendent_list

def api_get_json_batch(batch_url):
    """Get JSON results in batches of 500"""
    re_next_link = re.compile(r'<(.+)>; rel="next"')
    retries = Retry(total=5, backoff_factor=0.25, status_forcelist=[500, 502, 503, 504])
    session = requests.Session()
    session.mount("https://", HTTPAdapter(max_retries=retries))

    while batch_url:
        records_batch = session.get(batch_url)
        records_batch.raise_for_status()
        records_batch_json = records_batch.json()
        record_count = int(records_batch.headers["x-total-results"])

        yield records_batch_json, record_count

        if "Link" in records_batch.headers:
            match = re_next_link.match(records_batch.headers["Link"])
            batch_url = match.group(1)
        else:
            batch_url = None

def json_to_fasta(protein_record, tax_id_descendent_list, gene_name=None):
    """Convert JSON record to FASTA format"""
    # Check for required fields
    if 'sequence' not in protein_record or 'value' not in protein_record['sequence']:
        return None, None

    header = f"UPI|{protein_record['uniParcId']}"
    if gene_name:
        header += f"|{gene_name}"

    # Add UniProt accessions if available
    if 'uniProtKBAccessions' in protein_record and protein_record['uniProtKBAccessions']:
        header += f"|{protein_record['uniProtKBAccessions'][0]}"

    # Add creation date if available
    if 'oldestCrossRefCreated' in protein_record:
        header += f"|{protein_record['oldestCrossRefCreated']}"

    # Add gene name
    if gene_name:
        header += f" GN={gene_name}"

    sequence = protein_record['sequence']['value']
    return header, sequence

def download_proteins(tax_id, gene_list, raw_fasta):
    """Download proteins for given taxonomy ID and gene list"""
    print(f"\nStarting protein download with TaxID: {tax_id}")
    tax_id_descendent_list = api_taxid_descendent_list(tax_id)
    records_fasta_list = []
    total_count = 0

    for gene in gene_list:
        # First try exact taxonomy match
        url = f"https://rest.uniprot.org/uniparc/search?compressed=false&format=json&query=%28%28gene%3A{gene}%29%20AND%20%28taxonomy_id%3A{tax_id}%29%20NOT%20%28database%3AFusionGDB%29%29&size=500"
        print(f"\nRequesting URL: {url}")

        # Make initial request and print response info
        initial_response = requests.get(url)
        print(f"Response status code: {initial_response.status_code}")
        initial_data = initial_response.json()
        results = initial_data.get('results', [])
        print(f"Found {len(results)} results")

        # If no results, try with parent taxonomy (Homo for Homo sapiens)
        if not results:
            parent_taxid = 9605  # Homo genus
            url = f"https://rest.uniprot.org/uniparc/search?compressed=false&format=json&query=%28%28gene%3A{gene}%29%20AND%20%28taxonomy_id%3A{parent_taxid}%29%20NOT%20%28database%3AFusionGDB%29%29&size=500"
            print(f"\nNo results with species taxid, trying genus taxid: {parent_taxid}")
            print(f"New URL: {url}")
            initial_response = requests.get(url)
            initial_data = initial_response.json()
            results = initial_data.get('results', [])

        try:
            for record in results:
                header, sequence = json_to_fasta(record, tax_id_descendent_list, gene)
                if header and sequence:
                    record_fasta = SeqRecord(Seq(sequence),
                                           id=header.replace(",", "").replace(";", "").replace(":", ""),
                                           description="")
                    records_fasta_list.append(record_fasta)
                    total_count += 1
            print(f"  Downloaded {total_count} proteins so far...")

        except requests.exceptions.RequestException as e:
            print(f"Error downloading {gene}: {str(e)}")
            continue

    # Write results to file
    if records_fasta_list:
        with open(raw_fasta, "w") as output_handle:
            SeqIO.write(records_fasta_list, output_handle, "fasta")
        print(f"Successfully downloaded {total_count} proteins")
        return total_count
    return 0

def remove_duplicates(raw_fasta, output_folder):
    """Remove duplicate sequences from the FASTA file"""
    if not os.path.exists(raw_fasta):
        return 0

    records = list(SeqIO.parse(raw_fasta, "fasta"))
    unique_sequences = {}

    for record in records:
        seq = str(record.seq)
        if seq not in unique_sequences:
            unique_sequences[seq] = record

    # Create fasta_remove_redundancy folder
    redundancy_folder = os.path.join(output_folder, "fasta_remove_redundancy")
    os.makedirs(redundancy_folder, exist_ok=True)

    # Save unfiltered database
    shutil.copy(raw_fasta, os.path.join(redundancy_folder, "unfiltered_database.fasta"))

    # Write unique sequences
    filtered_path = os.path.join(redundancy_folder, "filtered_database.fasta")
    with open(filtered_path, "w") as handle:
        SeqIO.write(unique_sequences.values(), handle, "fasta")

    # Move filtered database to main folder
    output_path = os.path.join(output_folder, "filtered_mf.fasta")
    shutil.copy(filtered_path, output_path)

    return len(unique_sequences)

def generate_metadata(filtered_fasta, metadata_folder, gene_list_path):
    """Generate metadata files about the database"""
    os.makedirs(metadata_folder, exist_ok=True)

    # Read the FASTA file
    records = list(SeqIO.parse(filtered_fasta, "fasta"))

    # Generate basic stats
    with open(os.path.join(metadata_folder, "summary.txt"), "w") as f:
        f.write(f"Total sequences: {len(records)}\n")

        if gene_list_path:
            with open(gene_list_path) as gene_file:
                genes = [line.strip() for line in gene_file if line.strip()]
                f.write(f"Total genes searched: {len(genes)}\n")

                # Find which genes were found
                found_genes = set()
                for record in records:
                    for gene in genes:
                        if f"GN={gene}" in record.id:
                            found_genes.add(gene)

                f.write(f"Genes found: {len(found_genes)}\n")

                # Write genes not found
                not_found = set(genes) - found_genes
                if not_found:
                    with open(os.path.join(metadata_folder, "genes_NOT_retrieved.txt"), "w") as nf:
                        for gene in sorted(not_found):
                            nf.write(f"{gene}\n")

    # Generate species statistics
    species_data = {}
    for record in records:
        match = re.search(r"OS=([^OX]+)OX=(\d+)", record.id)
        if match:
            species = match.group(1).strip()
            taxid = match.group(2)
            if species not in species_data:
                species_data[species] = {"count": 0, "taxid": taxid}
            species_data[species]["count"] += 1

    with open(os.path.join(metadata_folder, "species_retrieved.csv"), "w") as f:
        f.write("Species,TaxID,Count\n")
        for species, data in species_data.items():
            f.write(f"{species},{data['taxid']},{data['count']}\n")

def run_integrated_proteoparc(tax_id, gene_list_path, output_folder):
    """Run the complete pipeline"""
    os.makedirs(output_folder, exist_ok=True)

    # Read gene list
    with open(gene_list_path) as f:
        gene_list = [line.strip() for line in f if line.strip()]

    # Step 1: Download proteins
    raw_fasta = os.path.join(output_folder, "raw_proteins.fasta")
    print("Downloading proteins...")
    num_downloaded = download_proteins(tax_id, gene_list, raw_fasta)

    if num_downloaded == 0:
        print("No proteins found")
        return

    print(f"Downloaded {num_downloaded} proteins")

    # Step 2: Remove duplicates
    print("Removing duplicates...")
    num_unique = remove_duplicates(raw_fasta, output_folder)
    print(f"Reduced to {num_unique} unique sequences")

    # Step 3: Generate metadata
    print("Generating metadata...")
    metadata_folder = os.path.join(output_folder, "metadata")
    generate_metadata(os.path.join(output_folder, "filtered_mf.fasta"),
                     metadata_folder,
                     gene_list_path)

    print(f"Pipeline completed. Results in: {output_folder}")

def run_proteoparc(b):
    """Run the ProteoParc pipeline"""
    if not hasattr(run_proteoparc, 'taxonomy_id'):
        print("Please select a taxonomy first")
        return

    paths = create_gene_list_file(b, taxonomy_dict, run_proteoparc.taxonomy_id) # Pass taxonomy_dict and taxonomy_id
    if paths:
        run_integrated_proteoparc(run_proteoparc.taxonomy_id, paths['genelist'], paths['root'])

def get_selected_tissue():
    """Determines active tissue type from selected genes"""
    selected_genes = get_selected_genes()
    tissue_counts = {}

    for gene in selected_genes:
        for tissue, genes in all_genes.items():
            if any(gene == g[0] for g in genes):
                tissue_counts[tissue] = tissue_counts.get(tissue, 0) + 1

    if not tissue_counts:
        return "mixed"
    return max(tissue_counts.items(), key=lambda x: x[1])[0]

def create_project_structure(taxonomy_dict, taxonomy_id, base_path="/content/drive/MyDrive/Colab_Notebooks/palaeoparc"):
    """
    Creates directory structure:
    base_path/
        tissue_type/
            taxa_name/
                YYYYMMDD_HHMMSS/
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    tissue_type = get_selected_tissue()
    taxa_name = taxonomy_dict.get(taxonomy_id, "unknown_taxonomy")

    # Sanitize names
    tissue_folder = tissue_type.lower().replace(" ", "_")
    taxa_folder = taxa_name.lower().replace(" ", "_")

    # Build paths
    timestamp_path = os.path.join(base_path, tissue_folder, taxa_folder, timestamp)
    os.makedirs(timestamp_path, exist_ok=True)
    os.makedirs(os.path.join(timestamp_path, "metadata"), exist_ok=True)

    # Generate file paths
    fasta_name = f"{tissue_folder}_{taxa_folder}.fasta"

    return {
        'root': timestamp_path,
        'fasta': os.path.join(timestamp_path, fasta_name),
        'genelist': os.path.join(timestamp_path, "genelist.txt"),
        'metadata': os.path.join(timestamp_path, "metadata")
    }


In [ ]:
#@title GUI
def create_taxonomy_buttons(taxonomy_dict):
    buttons = []
    for tax_id, name in taxonomy_dict.items():
        button = widgets.Button(
            description=name,
            tooltip=f"{name} (ID: {tax_id})",
            layout=widgets.Layout(width='auto', height='auto'),
            style={'button_color': '#e7e7e7', 'font_weight': 'normal'}
        )

        def on_button_click(b, tax_id=tax_id, name=name):
            global selected_taxonomy_id
            selected_taxonomy_id = tax_id
            output_dir = create_output_directory(taxonomy_dict, tax_id)
            create_gene_list_file.output_dir = output_dir  # Store output_dir for later use
            run_proteoparc.taxonomy_id = tax_id  # Store taxonomy_id for later use
            url = f"https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id={tax_id}"
            clear_output()
            display(HTML("<h2>Selected Taxonomy</h2>"))
            display(HTML(f"<h3>{name} (ID: {tax_id})</h3>"))
            display(HTML(f"<a href='{url}' target='_blank'>Open NCBI Taxonomy Browser</a>"))
            display(HTML(f"<p>Output directory: {output_dir}</p>"))
            display(HTML("<h2>Select Taxonomy</h2>"))
            display(button_grid)
            display(HTML("<h2>Gene Selection by Tissue</h2>"))
            display(widgets.HBox([select_all_button, deselect_all_button]))
            display(tissue_selection)
            display(widgets.HBox([print_button, save_button, run_proteoparc_button]))

        button.on_click(on_button_click)
        buttons.append(button)

    button_grid = widgets.GridBox(
        buttons,
        layout=widgets.Layout(
            grid_template_columns='repeat(3, auto)',
            grid_gap='10px',
            width='100%'
        )
    )

    return button_grid

def create_gene_checkboxes(gene_list):
    return [widgets.Checkbox(
        value=False,
        description=f"{gene} ({desc})",
        indent=False,
        layout=widgets.Layout(width='auto')
    ) for gene, desc in gene_list]


def create_tissue_selection(all_genes):
    tissue_checkboxes = {}
    gene_checkboxes = {}
    tissue_accordions = {}

    for tissue, genes in all_genes.items():
        tissue_checkbox = widgets.Checkbox(
            value=False,
            description=f"Select all {tissue} genes",
            indent=False,
            layout=widgets.Layout(width='auto')
        )
        tissue_checkboxes[tissue] = tissue_checkbox

        checkboxes = [widgets.Checkbox(description=gene[0]) for gene in genes] #create gene checkboxes
        gene_checkboxes.update({f"{tissue}_{gene[0]}": checkbox for gene, checkbox in zip(genes, checkboxes)})

        grid = widgets.GridBox(
            checkboxes,
            layout=widgets.Layout(
                grid_template_columns='repeat(3, minmax(200px, 1fr))',
                grid_gap='10px',
                width='100%'
            )
        )

        accordion = widgets.Accordion([grid], titles=[f"{tissue} Genes"])
        accordion.selected_index = None
        tissue_accordions[tissue] = accordion

        def on_tissue_change(change, t=tissue):
            if change['name'] == 'value':
                for gene, checkbox in gene_checkboxes.items():
                    if gene.startswith(f"{t}_"):
                        checkbox.value = change['new']
        tissue_checkbox.observe(on_tissue_change)

        def on_gene_change(change, t=tissue):
            tissue_checkbox.unobserve(on_tissue_change)
            tissue_checkbox.value = all(checkbox.value for gene, checkbox in gene_checkboxes.items() if gene.startswith(f"{t}_"))
            tissue_checkbox.observe(on_tissue_change)

        for gene, checkbox in gene_checkboxes.items():
            if gene.startswith(f"{tissue}_"):
                checkbox.observe(lambda change, t=tissue, c=checkbox: on_gene_change(change, t))

    tissue_vbox = widgets.VBox([
        widgets.HBox([tissue_checkboxes[tissue], tissue_accordions[tissue]])
        for tissue in all_genes.keys()
    ])

    return tissue_vbox, gene_checkboxes

def select_all(b):
    for checkbox in all_gene_checkboxes.values():
        checkbox.value = True

def deselect_all(b):
    for checkbox in all_gene_checkboxes.values():
        checkbox.value = False

def get_selected_genes():
    return [gene.split('_')[1] for gene, checkbox in all_gene_checkboxes.items() if checkbox.value]

def create_gene_list_file(b, taxonomy_dict, taxonomy_id):
    """Creates gene list file in organized directory structure"""
    selected_genes = get_selected_genes()
    if not selected_genes:
        print("No genes selected. Please select at least one gene.")
        return None

    paths = create_project_structure(taxonomy_dict, taxonomy_id)

    with open(paths['genelist'], 'w') as f:
        for gene in selected_genes:
            f.write(f"{gene}\n")

    print(f"Project directory created at: {paths['root']}")
    return paths

def create_output_directory(taxonomy_dict, taxonomy_id):
    name = taxonomy_dict.get(taxonomy_id, "Unknown_Taxonomy")
    # Calculate project_name here instead of using the undefined variable
    project_name = get_project_name(taxonomy_id, taxonomy_dict, "Unknown") # Assuming gene_type is unknown at this point
    output_dir = f"/content/drive/MyDrive/Colab Notebooks/proteoparc/{project_name}"
    os.makedirs(output_dir, exist_ok=True)
    return output_dir

def get_project_name(taxonomy_id, taxonomy_dict, gene_type):
    taxonomy_name = taxonomy_dict.get(taxonomy_id, "Unknown")
    return f"{taxonomy_name.lower().replace(' ', '_')}_{gene_type.lower()}_genes"

In [ ]:
#@title #Select genes and run the analysis

# Global variables for UI
all_gene_checkboxes = {}

# Create UI elements
tissue_selection, all_gene_checkboxes = create_tissue_selection(all_genes)
select_all_button = widgets.Button(description="Select All Genes")
deselect_all_button = widgets.Button(description="Deselect All Genes")
print_button = widgets.Button(description="Print Selected Genes")
save_button = widgets.Button(description="Save Gene List")
run_proteoparc_button = widgets.Button(description="Run ProteoParc")

# Attach functions to buttons
select_all_button.on_click(select_all)
deselect_all_button.on_click(deselect_all)
print_button.on_click(lambda b: print("Selected genes:", get_selected_genes()))
save_button.on_click(create_gene_list_file)
run_proteoparc_button.on_click(run_proteoparc)


# Create the button grid
button_grid = create_taxonomy_buttons(taxonomy_dict)

# Initial display
display(HTML("<h2>Select Taxonomy</h2>"))
display(button_grid)
display(HTML("<h2>Gene Selection by Tissue</h2>"))
display(widgets.HBox([select_all_button, deselect_all_button]))
display(tissue_selection)
display(widgets.HBox([print_button, save_button, run_proteoparc_button]))